In [1]:
import os
import site
import shutil
import distutils
import numpy as np
from pandas import read_csv
from pandas import read_html
from pandas import DataFrame as DF 
from pandas import date_range
from pandas.api import types as pd_dtype
from pandas import errors as pd_errors
from functools import reduce
from urllib import parse as urlparse
from bs4 import BeautifulSoup as BS
import time
import datetime
import warnings

g_drive_mod_path = r"/content/drive/MyDrive/Git Repos/Python/Custom_Modules"

# create custom modules folder in default pkg location
colab_mod_path = site.getsitepackages()[0]  # get path where packages are installed
colab_cust_mod_path = f"{colab_mod_path}/hakuna_patata_modules"

if os.path.exists(colab_cust_mod_path):
    shutil.rmtree(colab_cust_mod_path)
else:
    pass

try:
    shutil.copytree(g_drive_mod_path, colab_cust_mod_path)
    print(f"{os.path.basename(g_drive_mod_path)} successfully copied from {g_drive_mod_path} to {colab_cust_mod_path}")
except:
    print(f"Copy failed!")


from hakuna_patata_modules import hp_baseball_data
from hakuna_patata_modules import hp_sqlite
from google.colab import data_table

data_table.enable_dataframe_formatter()
data_table.DataTable.max_columns = 170
def DT(df, num_rows_per_page=10, max_cols=30, max_rows=None, include_index=None, min_width=None):
    from google.colab import data_table

def install_selenium():
    os.system('apt-get update')
    os.system('apt install chromium-chromedriver')
    os.system('pip install selenium')

install_selenium()

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
    

Custom_Modules successfully copied from /content/drive/MyDrive/Git Repos/Python/Custom_Modules to /usr/local/lib/python3.7/dist-packages/hakuna_patata_modules


In [ ]:
class MLBPredict:

    def __init__(self, dflt_dwnld_dir='/content/MLB_Predictor', src_db_dir='/content/drive/MyDrive/Git Repos/Python/MLB_Predictor'):
        
        ## Copy over existing DB if exists, else create appropriate project directories
        self.dflt_dwnld_dir = f'{os.getcwd()}/MLB_Predictor' if dflt_dwnld_dir is None or dflt_dwnld_dir == '' else dflt_dwnld_dir
        self.src_db_dir = src_db_dir
        if os.path.exists(self.src_db_dir):
            if os.path.exists(self.dflt_dwnld_dir):
                shutil.rmtree(self.dflt_dwnld_dir)
                shutil.copytree(src=self.src_db_dir, dst=self.dflt_dwnld_dir)
            else:
                shutil.copytree(src=self.src_db_dir, dst=self.dflt_dwnld_dir)
        else:
            os.mkdir(self.dflt_dwnld_dir)

        ## create selenium Chrome web driver
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        prefs = {'download.default_directory':self.dflt_dwnld_dir}
        options.add_experimental_option('prefs', prefs)
        self.driver = webdriver.Chrome('chromedriver', options=options)

        self.db_path = f'{self.dflt_dwnld_dir}/MLB_DATA.db'
        self.src_db_path = f'{self.src_db_dir}/MLB_DATA.db'
        self.player_id_table = 'PLAYER_ID_LINKS'
        self.team_id_table = 'TEAM_ID_LINKS'
        self.batter_gamelog_table = 'BATTER_GAMELOG_STATS'
        self.pitcher_gamelog_table = 'PITCHER_GAMELOG_STATS'
        self.no_data_dates_table = 'NO_DATA_DATES'
        self.conn, self.conn_c = hp_sqlite.sqlite_conn_cursor(self.db_path)



    def selenium_chrome_driver(self):
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')

        if not os.path.exists(self.dflt_dwnld_dir):
            os.mkdir(self.dflt_dwnld_dir)
        else:
            pass

        prefs = {'download.default_directory':self.dflt_dwnld_dir}
        options.add_experimental_option('prefs', prefs)

        return webdriver.Chrome('chromedriver', options=options)



    @classmethod
    def col_name_format(cls, cols):
        format_cols = []
        blank_name_count = 1

        for col in cols:
            if col == '#':
                col = 'NUMBER_LINE'
            else:
                pass
            col = col.strip()
            col = col.upper()
            col = col.replace('1B', 'SINGLE')
            col = col.replace('2B', 'DOUBLE')
            col = col.replace('3B', 'TRIPLE')
            col = col.replace('.', '_')
            col = col.replace('-', '_')
            col = col.replace('%', '_PERCENT')
            col = col.replace('/', '_PER_')
            col = col.replace('+', '_PLUS')
            col = col.replace('#', 'NUMBER_LINE')
            col = col.replace(' (PI)', '_PI')
            col = col.replace(' (SC)', '_SC')
            col = col.replace(' ', '')

            if col == '':
                col = f'NULL_COL_{blank_name_count}'
                blank_name_count += 1
            else:
                pass

            format_cols.append(col)

        return format_cols


    
    @classmethod
    def null_col_remove(cls, df, startswith='NULL_COL'):
        return df.loc[:,~df.columns.str.contains('NULL_COL')]



    @classmethod
    def df_dtype_convert(cls, df, round_str_2_dec=5):
        for col in df.columns:
            try:
                if sum(df[col].astype('int') != df[col]) > 0:
                    df[col] = df[col].astype('int')
                else:
                    df[col] = df[col].astype('float')
            except:
                pass

            try:
                df[col] = round(df[col].str.rstrip('%').astype('float') / 100.0, round_str_2_dec)
            except: 
                pass 

        return df


    
    @classmethod
    def df_sqlite_dtype_convert(cls, df):
        dtype_dict = {}
        for col in df.columns:
            if pd_dtype.is_integer_dtype(df[col]):
                dtype_dict[col] = 'INTEGER'

            elif pd_dtype.is_float_dtype(df[col]):
                dtype_dict[col] = 'REAL'

            else:
                dtype_dict[col] = 'TEXT'

        return dtype_dict



    @classmethod
    def mlb_dates(cls, start_year='2015', end_year=None):

        if int(start_year) < 2015:
            print('ERROR: No StatCast data prior to 2015. Enter year >= 2015.')
        else:
            start_year = str(int(start_year))

        if end_year is None:
            end_year = str(datetime.date.today().year)
        elif int(end_year) < int(start_year):
            print('ERROR: End Year must be >= Start Year!')
        else:
            end_year = str(end_year)

        all_dates = date_range(f'{start_year}-01-01', f'{end_year}-12-31')
        dt_range = []
        for dt in all_dates:
            if dt.month >= 3 and dt.month <= 10 and dt < datetime.date.today():
                dt_range.append(dt.strftime('%Y-%m-%d'))
            else:
                pass
                
        return dt_range



    def save_db(self, auto=False):

        src = self.db_path
        dst = self.src_db_path


        if auto:
            save_prompt='Y'
        else:
            print(f'SOURCE FILE: \t\t{src}')
            print(f'DESTINATION FILE: \t{dst}')
            save_prompt = input(f'Overwrite destination file with source file if exists? (Y/N): ')

        if save_prompt.upper()=='Y':
            if os.path.exists(src):
                try:
                    shutil.copy(src, dst)
                    if auto:
                        pass
                    else:
                        print('DB successfully saved!')
                except Exception as e:
                    print(e)
            else:
                print(f'{src} file does not exist! No files saved.')

        else:
            print(f'Save terminated. No files saved.')


    def update_player_id_rows(self, start_year='2015', end_year=None):

        with warnings.catch_warnings():
            warnings.simplefilter(action='ignore', category=FutureWarning)
            if int(start_year) < 2015:
                print('ERROR: No StatCast data prior to 2015. Enter year >= 2015.')
            else:
                start_year = str(int(start_year))

            if end_year is None:
                end_year = str(datetime.date.today().year)
            elif int(end_year) < int(start_year):
                print('ERROR: End Year must be >= Start Year!')
            else:
                end_year = str(end_year)

            batter_url = f'https://www.fangraphs.com/leaders.aspx?pos=all&stats=bat&lg=all&qual=1&type=c%2C0&season={end_year}&month=0&season1={start_year}&ind=0'
            pitcher_url = f'https://www.fangraphs.com/leaders.aspx?pos=all&stats=pit&lg=all&qual=1&type=c%2C0&season={end_year}&month=0&season1={start_year}&ind=0'

            pos_urls = {
                'B':batter_url
                , 'P':pitcher_url
            }

            for pos, url in pos_urls.items():
                self.driver.get(url)

                delay = 10
                try:
                    dwnld_element = WebDriverWait(self.driver, delay).until(EC.presence_of_element_located((By.ID, 'LeaderBoard1_cmdCSV')))
                    self.driver.execute_script("arguments[0].click();", dwnld_element)
                except:
                    print(f'TimeoutException: Download element step.')


                for i in range(1,21):
                    if os.path.exists(f'{self.dflt_dwnld_dir}/FanGraphs Leaderboard.csv'):
                        os.rename(f"{self.dflt_dwnld_dir}/FanGraphs Leaderboard.csv", f"{self.dflt_dwnld_dir}/PlayerData.csv")
                        break
                    else:
                        if i == 20:
                            print('TimeoutException: File renaming step.')
                            break
                        else:
                            time.sleep(.5)

                for i in range(1,21):
                    if os.path.exists(f'{self.dflt_dwnld_dir}/PlayerData.csv'):
                        player_df = read_csv(f'{self.dflt_dwnld_dir}/PlayerData.csv')
                        break
                    else:
                        if i == 20:
                            print('TimeoutException: CSV to DataFrame step.')
                            break
                        else:
                            time.sleep(.5)


                player_df = player_df[['Name', 'playerid']]
                player_df.columns = ['NAME', 'PLAYERID']

                player_df['NAMEID'] = player_df['NAME'].str.lower().str.replace(' ', '-').str.replace('.', '')
                player_df['PLAYER_URL'] = 'https://www.fangraphs.com/statss.aspx?playerid=' + player_df['PLAYERID'].astype('string')
                player_df['PLAYER_GAMELOG_STANDARD_URL'] = 'https://www.fangraphs.com/players/' + player_df['NAMEID'] + '/' + player_df['PLAYERID'].astype('string') + f'/game-log?season=all&type=1&position={pos}'
                player_df['PLAYER_GAMELOG_ADVANCED_URL'] = 'https://www.fangraphs.com/players/' + player_df['NAMEID'] + '/' + player_df['PLAYERID'].astype('string') + f'/game-log?season=all&type=2&position={pos}'
                player_df['PLAYER_GAMELOG_STATCAST_URL'] = 'https://www.fangraphs.com/players/' + player_df['NAMEID'] + '/' + player_df['PLAYERID'].astype('string') + f'/game-log?season=all&type=24&position={pos}'
                player_df['PLAYER_GAMELOG_BATTED_BALL_URL'] = 'https://www.fangraphs.com/players/' + player_df['NAMEID'] + '/' + player_df['PLAYERID'].astype('string') + f'/game-log?season=all&type=3&position={pos}'
                player_df['PLAYER_GAMELOG_MORE_BB_URL'] = 'https://www.fangraphs.com/players/' + player_df['NAMEID'] + '/' + player_df['PLAYERID'].astype('string') + f'/game-log?season=all&type=4&position={pos}'
                player_df['PLAYER_GAMELOG_PLATE_DISCIPLINE_URL'] = 'https://www.fangraphs.com/players/' + player_df['NAMEID'] + '/' + player_df['PLAYERID'].astype('string') + f'/game-log?season=all&type=8&position={pos}'
                player_df['POS'] = pos

                col_dtypes = self.df_sqlite_dtype_convert(player_df)

                try:
                    db_player_df = hp_sqlite.sql_to_df(self.db_path, f'SELECT * FROM {self.player_id_table}')  # get values from DB table
                    player_df = player_df.astype(str)
                    new_row_mask = player_df.merge(db_player_df.astype(str), how='left', left_on=['PLAYERID','POS'], right_on=['PLAYERID','POS'], indicator=True)['_merge']=='left_only'  # create mask to filter on new rows not already in DB table
                    new_rows_df = player_df[new_row_mask]  # filter on new rows not already in DB table
                    new_rows_df.to_sql(self.player_id_table, con=self.conn, if_exists='append', index=False, chunksize=1000, method='multi', dtype=col_dtypes)
                    print(f'{self.player_id_table}\n{"-"*20}\nTABLE STATUS: \tEXISTING\n{"="*30}\nROWS RETRIEVED: \t{len(player_df)}\nEXISTING IN DB: \t{len(player_df) - len(new_rows_df)}\nNEW ROWS INSERTED: \t{len(new_rows_df)}\n\n')

                except:
                    player_df.to_sql(self.player_id_table, con=self.conn, if_exists='fail', index=False, chunksize=1000, method='multi', dtype=col_dtypes)  # if table does not exist, create a new table
                    print(f'{self.player_id_table}\n{"-"*20}\nTABLE STATUS: \tNEW TABLE\n{"="*30}\nROWS INSERTED: \t{len(player_df)}\n\n')
                    
                ## cleanup
                os.remove(f'{self.dflt_dwnld_dir}/PlayerData.csv')



    def update_team_id_rows(self, start_year='2015', end_year=None):

        with warnings.catch_warnings():
            warnings.simplefilter(action='ignore', category=FutureWarning)
            if int(start_year) < 2015:
                print('ERROR: No StatCast data prior to 2015. Enter year >= 2015.')
            else:
                start_year = str(int(start_year))

            if end_year is None:
                end_year = str(datetime.date.today().year)
            elif int(end_year) < int(start_year):
                print('ERROR: End Year must be >= Start Year!')
            else:
                end_year = str(end_year)

            url = f'https://www.fangraphs.com/leaders.aspx?pos=all&stats=bat&lg=all&qual=0&type=8&season={end_year}&month=0&season1={start_year}&team=0,ts'

            self.driver.get(url)

            self.driver.get(url)

            delay = 10
            xpath = '//*[@id="LeaderBoard1_dg1_ctl00"]/tbody/tr/td[2]/a'

            try:
                WebDriverWait(self.driver, delay).until(EC.presence_of_element_located((By.XPATH, xpath)))
            except:
                print(f'TimeoutException: Scrape step.')

            team_name_list = []
            team_id_list = []
            for team in self.driver.find_elements(By.XPATH, xpath):
                team_name = team.text
                team_id = urlparse.parse_qs(urlparse.urlparse(team.get_attribute('href')).query)['team'][0]
                team_name_list.append(team_name)
                team_id_list.append(team_id)

            team_df = DF(zip(team_id_list, team_name_list), columns=['TEAMID', 'NAME'])

            col_dtypes = self.df_sqlite_dtype_convert(team_df)

            try:
                db_team_df = hp_sqlite.sql_to_df(self.db_path, f'SELECT * FROM {self.team_id_table}')  # get values from DB table
                team_df = team_df.astype(str)
                new_row_mask = team_df.merge(db_team_df.astype(str), how='left', left_on='TEAMID', right_on='TEAMID', indicator=True)['_merge']=='left_only'  # create mask to filter on new rows not already in DB table
                new_rows_df = team_df[new_row_mask]  # filter on new rows not already in DB table
                new_rows_df.to_sql(self.team_id_table, con=self.conn, if_exists='append', index=False, chunksize=1000, method='multi', dtype=col_dtypes)
                print(f'{self.team_id_table}\n{"-"*20}\nTABLE STATUS: \tEXISTING\n{"="*30}\nROWS RETRIEVED: \t{len(team_df)}\nEXISTING IN DB: \t{len(team_df) - len(new_rows_df)}\nNEW ROWS INSERTED: \t{len(new_rows_df)}\n\n')

            except:
                team_df.to_sql(self.team_id_table, con=self.conn, if_exists='fail', index=False, chunksize=1000, method='multi', dtype=col_dtypes)  # if table does not exist, create a new table
                print(f'{self.team_id_table}\n{"-"*20}\nTABLE STATUS: \tNEW TABLE\n{"="*30}\nROWS INSERTED: \t{len(team_df)}\n\n')


    def update_gamelog_rows(self, start_year='2015', end_year=None):
        with warnings.catch_warnings():
            warnings.simplefilter(action='ignore', category=pd_errors.PerformanceWarning)

            if int(start_year) < 2015:
                print('ERROR: No StatCast data prior to 2015. Enter year >= 2015.')
            else:
                start_year = str(int(start_year))

            if end_year is None:
                end_year = str(datetime.date.today().year)
            elif int(end_year) < int(start_year):
                print('ERROR: End Year must be >= Start Year!')
            else:
                end_year = str(end_year)


            row1_xpath = '//*[@id="LeaderBoard1_dg1_ctl00__0"]/td[1]'
            no_data_class = 'rgNoRecords'
            elem_id = 'LeaderBoard1_cmdCSV'
            batter_stats = 'c%2c3%2c4%2c5%2c6%2c7%2c8%2c9%2c10%2c11%2c12%2c13%2c14%2c15%2c16%2c17%2c18%2c19%2c20%2c21%2c22%2c23%2c24%2c25%2c26%2c27%2c28%2c29%2c30%2c31%2c32%2c33%2c34%2c35%2c36%2c37%2c38%2c39%2c40%2c41%2c42%2c43%2c44%2c45%2c46%2c47%2c48%2c49%2c50%2c51%2c52%2c53%2c54%2c55%2c56%2c57%2c58%2c59%2c60%2c61%2c62%2c63%2c64%2c65%2c66%2c67%2c68%2c69%2c70%2c71%2c72%2c73%2c74%2c75%2c76%2c77%2c78%2c79%2c80%2c81%2c82%2c83%2c84%2c85%2c86%2c87%2c88%2c89%2c90%2c91%2c92%2c93%2c94%2c95%2c96%2c97%2c98%2c99%2c100%2c101%2c102%2c103%2c104%2c105%2c106%2c107%2c108%2c109%2c110%2c111%2c112%2c113%2c114%2c115%2c116%2c117%2c118%2c119%2c120%2c121%2c122%2c123%2c124%2c125%2c126%2c127%2c128%2c129%2c130%2c131%2c132%2c133%2c134%2c135%2c136%2c137%2c138%2c139%2c140%2c141%2c142%2c143%2c144%2c145%2c146%2c147%2c148%2c149%2c150%2c151%2c152%2c153%2c154%2c155%2c156%2c157%2c158%2c159%2c160%2c161%2c162%2c163%2c164%2c165%2c166%2c167%2c168%2c169%2c170%2c171%2c172%2c173%2c174%2c175%2c176%2c177%2c178%2c179%2c180%2c181%2c182%2c183%2c184%2c185%2c186%2c187%2c188%2c189%2c190%2c191%2c192%2c193%2c194%2c195%2c196%2c197%2c198%2c199%2c200%2c201%2c202%2c203%2c204%2c205%2c206%2c207%2c208%2c209%2c210%2c211%2c212%2c213%2c214%2c215%2c216%2c217%2c218%2c219%2c220%2c221%2c222%2c223%2c224%2c225%2c226%2c227%2c228%2c229%2c230%2c231%2c232%2c233%2c234%2c235%2c236%2c237%2c238%2c239%2c240%2c241%2c242%2c243%2c244%2c245%2c246%2c247%2c248%2c249%2c250%2c251%2c252%2c253%2c254%2c255%2c256%2c257%2c258%2c259%2c260%2c261%2c262%2c263%2c264%2c265%2c266%2c267%2c268%2c269%2c270%2c271%2c272%2c273%2c274%2c275%2c276%2c277%2c278%2c279%2c280%2c281%2c282%2c283%2c284%2c285%2c286%2c287%2c288%2c289%2c290%2c291%2c292%2c293%2c294%2c295%2c296%2c297%2c298%2c299%2c300%2c301%2c302%2c303%2c304%2c305%2c306%2c307%2c308%2c309%2c310%2c311%2c312%2c313%2c314%2c315%2c316%2c317%2c318'
            pitcher_stats = 'c%2c3%2c4%2c5%2c6%2c7%2c8%2c9%2c10%2c11%2c12%2c13%2c14%2c15%2c16%2c17%2c18%2c19%2c20%2c21%2c22%2c23%2c24%2c25%2c26%2c27%2c28%2c29%2c30%2c31%2c32%2c33%2c34%2c35%2c36%2c37%2c38%2c39%2c40%2c41%2c42%2c43%2c44%2c45%2c46%2c47%2c48%2c49%2c50%2c51%2c52%2c53%2c54%2c55%2c56%2c57%2c58%2c59%2c60%2c61%2c62%2c63%2c64%2c65%2c66%2c67%2c68%2c69%2c70%2c71%2c72%2c73%2c74%2c75%2c76%2c77%2c78%2c79%2c80%2c81%2c82%2c83%2c84%2c85%2c86%2c87%2c88%2c89%2c90%2c91%2c92%2c93%2c94%2c95%2c96%2c97%2c98%2c99%2c100%2c101%2c102%2c103%2c104%2c105%2c106%2c107%2c108%2c109%2c110%2c111%2c112%2c113%2c114%2c115%2c116%2c117%2c118%2c119%2c120%2c121%2c122%2c123%2c124%2c125%2c126%2c127%2c128%2c129%2c130%2c131%2c132%2c133%2c134%2c135%2c136%2c137%2c138%2c139%2c140%2c141%2c142%2c143%2c144%2c145%2c146%2c147%2c148%2c149%2c150%2c151%2c152%2c153%2c154%2c155%2c156%2c157%2c158%2c159%2c160%2c161%2c162%2c163%2c164%2c165%2c166%2c167%2c168%2c169%2c170%2c171%2c172%2c173%2c174%2c175%2c176%2c177%2c178%2c179%2c180%2c181%2c182%2c183%2c184%2c185%2c186%2c187%2c188%2c189%2c190%2c191%2c192%2c193%2c194%2c195%2c196%2c197%2c198%2c199%2c200%2c201%2c202%2c203%2c204%2c205%2c206%2c207%2c208%2c209%2c210%2c211%2c212%2c213%2c214%2c215%2c216%2c217%2c218%2c219%2c220%2c221%2c222%2c223%2c224%2c225%2c226%2c227%2c228%2c229%2c230%2c231%2c232%2c233%2c234%2c235%2c236%2c237%2c238%2c239%2c240%2c241%2c242%2c243%2c244%2c245%2c246%2c247%2c248%2c249%2c250%2c251%2c252%2c253%2c254%2c255%2c256%2c257%2c258%2c259%2c260%2c261%2c262%2c263%2c264%2c265%2c266%2c267%2c268%2c269%2c270%2c271%2c272%2c273%2c274%2c275%2c276%2c277%2c278%2c279%2c280%2c281%2c282%2c283%2c284%2c285%2c286%2c287%2c288%2c289%2c290%2c291%2c292%2c293%2c294%2c295%2c296%2c297%2c298%2c299%2c300%2c301%2c302%2c303%2c304%2c305%2c306%2c307%2c308%2c309%2c310%2c311%2c312%2c313%2c314%2c315%2c316%2c317%2c318%2c319%2c320%2c321%2c322%2c323%2c324%2c325%2c326%2c327%2c328%2c329%2c330%2c331%2c332'
            pos_stats = {
                'bat':batter_stats
                , 'pit':pitcher_stats
            }

            for pos, stats in pos_stats.items():
                if pos=='bat':
                    table_name = self.batter_gamelog_table
                    player_table_pos = 'B'
                elif pos=='pit':
                    table_name = self.pitcher_gamelog_table
                    player_table_pos = 'P'

                all_dates = self.mlb_dates(start_year=start_year, end_year=end_year)

                try:
                    db_gamelog_df = hp_sqlite.sql_to_df(self.db_path, f"SELECT DISTINCT DATE FROM {table_name}")  # get values from DB table
                    db_gamelog_dt_list = list(db_gamelog_df['DATE'])
                except:
                    db_gamelog_dt_list = []

                try:
                    db_no_data_date_df = hp_sqlite.sql_to_df(self.db_path, f"SELECT DISTINCT DATE FROM {self.no_data_dates_table}")  # get values from DB table
                    db_no_dt_list = list(db_no_data_date_df['DATE'])
                except:
                    db_no_dt_list = []

                exclude_dts = list(set(db_gamelog_dt_list + db_no_dt_list))

                dt_range = []
                for dt in all_dates:
                    if dt not in exclude_dts:
                        dt_range.append(dt)
                    else:
                        pass 


                for dt in dt_range:
                    url = f'https://www.fangraphs.com/leaders.aspx?pos=all&stats={pos}&lg=all&qual=1&type={stats}&month=1000&team=0&startdate={dt}&enddate={dt}&page=1_5000'

                    self.driver.get(url)

                    try:
                        no_data_text = self.driver.find_element(By.CLASS_NAME, no_data_class).text
                        try:
                            db_no_date_df = hp_sqlite.sql_to_df(self.db_path, f"SELECT DISTINCT DATE FROM {self.no_data_dates_table} WHERE DATE = '{dt}'")
                            if len(db_no_date_df) == 0:
                                raise LookupError()
                            print(f'{dt}: No data!')
                        except:
                            no_date_df = DF({'DATE':[dt]})
                            no_date_df.to_sql(self.no_data_dates_table, con=self.conn, if_exists='append', index=False, chunksize=1000, method='multi')
                            print(f'{dt}: No data! Date appended to {self.no_data_dates_table}')
                    except:
                        for _ in range(10):
                            try:
                                row1_text = self.driver.find_element(By.XPATH, row1_xpath).text
                                elem = self.driver.execute_script(f"return document.getElementById('{elem_id}');")
                                self.driver.execute_script("arguments[0].click();", elem)
                                for read_csv_try in range(10):
                                    if os.path.exists(f'{self.dflt_dwnld_dir}/FanGraphs Leaderboard.csv'):
                                        gamelog_df = read_csv(f'{self.dflt_dwnld_dir}/FanGraphs Leaderboard.csv')
                                        try:
                                            gamelog_df.columns = gamelog_df.columns.droplevel(0)
                                        except:
                                            pass
                                        gamelog_df.columns = self.col_name_format(gamelog_df.columns)
                                        try:
                                            gamelog_df = gamelog_df[gamelog_df['NUMBER_LINE'] == pd_dtype.is_integer]  # exclude rows commenting page size or no records to display
                                        except:
                                            pass
                                        gamelog_df = self.df_dtype_convert(gamelog_df)
                                        gamelog_df['DATE'] = dt

                                        col_dtypes = self.df_sqlite_dtype_convert(gamelog_df)

                                        try:
                                            db_gamelog_df = hp_sqlite.sql_to_df(self.db_path, f'SELECT * FROM {table_name}')  # get values from DB table
                                            gamelog_df = gamelog_df.astype(str)
                                            new_row_mask = gamelog_df.merge(db_gamelog_df.astype(str), how='left', left_on='DATE', right_on='DATE', indicator=True)['_merge']=='left_only'  # create mask to filter on new rows not already in DB table
                                            new_rows_df = gamelog_df[new_row_mask]  # filter on new rows not already in DB table
                                            new_rows_df.to_sql(table_name, con=self.conn, if_exists='append', index=False, chunksize=1000, method='multi', dtype=col_dtypes)
                                            print(f'{dt}: {self.db_path}..{table_name} - {len(new_rows_df)} rows inserted')

                                        except:
                                            gamelog_df.to_sql(table_name, con=self.conn, if_exists='fail', index=False, chunksize=1000, method='multi', dtype=col_dtypes)  # if table does not exist, create a new table
                                            print(f'{dt}: {self.db_path}..{table_name} - {len(gamelog_df)} rows inserted')

                                        os.remove(f'{self.dflt_dwnld_dir}/FanGraphs Leaderboard.csv')
                                        self.save_db(auto=True)
                                        break
                                    else:
                                        time.sleep(1)
                                break

                            except:
                                time.sleep(1)


In [ ]:
mlb = MLBPredict()


In [ ]:
mlb.update_gamelog_rows()


2022-09-01: /content/MLB_Predictor/MLB_DATA.db..BATTER_GAMELOG_STATS - 163 rows inserted
2022-09-02: /content/MLB_Predictor/MLB_DATA.db..BATTER_GAMELOG_STATS - 307 rows inserted
2022-09-03: /content/MLB_Predictor/MLB_DATA.db..BATTER_GAMELOG_STATS - 301 rows inserted
2022-09-04: /content/MLB_Predictor/MLB_DATA.db..BATTER_GAMELOG_STATS - 311 rows inserted
2022-09-05: /content/MLB_Predictor/MLB_DATA.db..BATTER_GAMELOG_STATS - 224 rows inserted
2022-09-06: /content/MLB_Predictor/MLB_DATA.db..BATTER_GAMELOG_STATS - 278 rows inserted
2018-10-01: /content/MLB_Predictor/MLB_DATA.db..PITCHER_GAMELOG_STATS - 11 rows inserted
2019-03-20: /content/MLB_Predictor/MLB_DATA.db..PITCHER_GAMELOG_STATS - 8 rows inserted
2019-03-21: /content/MLB_Predictor/MLB_DATA.db..PITCHER_GAMELOG_STATS - 12 rows inserted
2019-03-28: /content/MLB_Predictor/MLB_DATA.db..PITCHER_GAMELOG_STATS - 90 rows inserted
2019-03-29: /content/MLB_Predictor/MLB_DATA.db..PITCHER_GAMELOG_STATS - 65 rows inserted
2019-03-30: /content/M

KeyboardInterrupt: ignored

In [ ]:
hp_sqlite.sql_to_df(mlb.src_db_path, "SELECT MAX(DATE) FROM PITCHER_GAMELOG_STATS LIMIT 1000").iloc[0][0]


'2021-06-08'